## Load the data

In [1]:
from dagster import AssetKey
from pudl.etl import defs

demand_forecasts = defs.load_asset_value(AssetKey("raw_phmsagas__yearly_distribution"))

No dagster instance configuration file (dagster.yaml) found at /Users/sam/Documents/pudl-data/dagster_home. Defaulting to loading and storing all metadata with /Users/sam/Documents/pudl-data/dagster_home. If this is the desired behavior, create an empty dagster.yaml file in /Users/sam/Documents/pudl-data/dagster_home.
2024-09-21 12:44:41 -0400 - dagster - DEBUG - system - Loading file from: /Users/sam/Documents/pudl-data/dagster_home/storage/raw_phmsagas__yearly_distribution using PickledObjectFilesystemIOManager...


## Data exploration

In [2]:
demand_forecasts.head()

,additional_information,all_known_leaks_scheduled_for_repair,all_leaks_construction_defect_mains,all_leaks_construction_defect_services,all_leaks_corrosion_failure_mains,all_leaks_corrosion_failure_services,all_leaks_equipment_failure_mains,all_leaks_equipment_failure_services,all_leaks_excavation_damage_mains,all_leaks_excavation_damage_services,all_leaks_incorrect_operation_mains,all_leaks_incorrect_operation_services,all_leaks_material_defect_mains,all_leaks_material_defect_services,all_leaks_natural_force_damage_mains,all_leaks_natural_force_damage_services,all_leaks_other_mains,all_leaks_other_outside_force_mains,all_leaks_other_outside_force_services,all_leaks_other_services,all_leaks_outside_force_mains,all_leaks_outside_force_services,all_leaks_pipe_weld_joint_failure_mains,all_leaks_pipe_weld_joint_failure_services,all_leaks_third_party_mains,all_leaks_third_party_services,average_service_length_feet,commodity,correction_date,data_date,data_maturity,excavation_damage_excavation_practices,excavation_damage_locating_practices,excavation_damage_one_call_notification,excavation_damage_other,excavation_damage_total,excavation_tickets,federal_land_leaks_repaired_or_scheduled,filing_date,form_revision,hazardous_leaks_corrosion_failure_mains,hazardous_leaks_corrosion_failure_services,hazardous_leaks_equipment_failure_mains,hazardous_leaks_equipment_failure_services,hazardous_leaks_excavation_damage_mains,hazardous_leaks_excavation_damage_services,hazardous_leaks_incorrect_operation_mains,hazardous_leaks_incorrect_operation_services,hazardous_leaks_mechanical_joint_failure,hazardous_leaks_natural_force_damage_mains,hazardous_leaks_natural_force_damage_services,hazardous_leaks_other_mains,hazardous_leaks_other_outside_force_mains,hazardous_leaks_other_outside_force_services,hazardous_leaks_other_services,hazardous_leaks_pipe_weld_joint_failure_mains,hazardous_leaks_pipe_weld_joint_failure_services,headquarters_address_city,headquarters_address_county,headquarters_address_state,headquarters_address_street,headquarters_address_zip,log,main_1940s_miles,main_1950s_miles,main_1960s_miles,main_1970s_miles,main_1980s_miles,main_1990s_miles,main_2000s_miles,main_2010s_miles,main_2020s_miles,main_abs_2_in_or_less_miles,main_abs_2_to_4_in_miles,main_abs_4_to_8_in_miles,main_abs_8_to_12_in_miles,main_abs_over_12_in_miles,main_abs_total_miles,main_abs_unknown_miles,main_all_materials_2_in_or_less_miles,main_all_materials_2_to_4_in_miles,main_all_materials_4_to_8_in_miles,main_all_materials_8_to_12_in_miles,main_all_materials_over_12_in_miles,main_all_materials_total_miles,main_all_materials_unknown_miles,main_all_time_miles,main_cast_or_wrought_iron_2_in_or_less_miles,main_cast_or_wrought_iron_2_to_4_in_miles,main_cast_or_wrought_iron_4_to_8_in_miles,main_cast_or_wrought_iron_8_to_12_in_miles,main_cast_or_wrought_iron_miles,main_cast_or_wrought_iron_over_12_in_miles,main_cast_or_wrought_iron_total_miles,main_cast_or_wrought_iron_unknown_miles,main_cathodically_protected_steel_bare_miles,main_cathodically_protected_steel_coated_miles,main_copper_2_in_or_less_miles,main_copper_2_to_4_in_miles,main_copper_4_to_8_in_miles,main_copper_8_to_12_in_miles,main_copper_miles,main_copper_over_12_in_miles,main_copper_total_miles,main_copper_unknown_miles,main_ductile_iron_2_in_or_less_miles,main_ductile_iron_2_to_4_in_miles,main_ductile_iron_4_to_8_in_miles,main_ductile_iron_8_to_12_in_miles,main_ductile_iron_miles,main_ductile_iron_over_12_in_miles,main_ductile_iron_total_miles,main_ductile_iron_unknown_miles,main_other_2_in_or_less_miles,main_other_2_to_4_in_miles,main_other_4_to_8_in_miles,main_other_8_to_12_in_miles,main_other_alt_2_in_or_less_miles,main_other_alt_2_to_4_in_miles,main_other_alt_4_to_8_in_miles,main_other_alt_8_to_12_in_miles,main_other_alt_miles,main_other_alt_over_12_in_miles,main_other_alt_total_miles,main_other_alt_unknown_miles,main_other_material_detail_miles,main_other_miles,main_other_over_12_in_miles,main_other_plastic_2_in

In [24]:
def sorted_value_printer(column):
    # Assuming 'demand_forecasts' is your DataFrame
    sorted_values = sorted(demand_forecasts[column])

    # Pretty print the sorted columns
    for val in sorted_values:
        print(val)

In [11]:
print(demand_forecasts.filing_date.max())
print(demand_forecasts.filing_date.min())

print(demand_forecasts.report_date.max())
print(demand_forecasts.report_date.min())

print(demand_forecasts.report_year.max())
print(demand_forecasts.report_year.min())

2023-12-21 14:41:38
2022-10-28 12:52:08
2023-12-11 16:19:06
2022-10-28 12:52:08
2022.0
2022.0


In [30]:
demand_forecasts[demand_forecasts.operator_id_phmsa==383][['operator_id_phmsa','operator_name_phmsa']]

,operator_id_phmsa,operator_name_phmsa
17,383.0,AMARILLO NATURAL GAS INC
18,383.0,AMARILLO NATURAL GAS INC


In [27]:
demand_forecasts[demand_forecasts.operator_id_phmsa==383.0].head()

,additional_information,all_known_leaks_scheduled_for_repair,all_leaks_construction_defect_mains,all_leaks_construction_defect_services,all_leaks_corrosion_failure_mains,all_leaks_corrosion_failure_services,all_leaks_equipment_failure_mains,all_leaks_equipment_failure_services,all_leaks_excavation_damage_mains,all_leaks_excavation_damage_services,all_leaks_incorrect_operation_mains,all_leaks_incorrect_operation_services,all_leaks_material_defect_mains,all_leaks_material_defect_services,all_leaks_natural_force_damage_mains,all_leaks_natural_force_damage_services,all_leaks_other_mains,all_leaks_other_outside_force_mains,all_leaks_other_outside_force_services,all_leaks_other_services,all_leaks_outside_force_mains,all_leaks_outside_force_services,all_leaks_pipe_weld_joint_failure_mains,all_leaks_pipe_weld_joint_failure_services,all_leaks_third_party_mains,all_leaks_third_party_services,average_service_length_feet,commodity,correction_date,data_date,data_maturity,excavation_damage_excavation_practices,excavation_damage_locating_practices,excavation_damage_one_call_notification,excavation_damage_other,excavation_damage_total,excavation_tickets,federal_land_leaks_repaired_or_scheduled,filing_date,form_revision,hazardous_leaks_corrosion_failure_mains,hazardous_leaks_corrosion_failure_services,hazardous_leaks_equipment_failure_mains,hazardous_leaks_equipment_failure_services,hazardous_leaks_excavation_damage_mains,hazardous_leaks_excavation_damage_services,hazardous_leaks_incorrect_operation_mains,hazardous_leaks_incorrect_operation_services,hazardous_leaks_mechanical_joint_failure,hazardous_leaks_natural_force_damage_mains,hazardous_leaks_natural_force_damage_services,hazardous_leaks_other_mains,hazardous_leaks_other_outside_force_mains,hazardous_leaks_other_outside_force_services,hazardous_leaks_other_services,hazardous_leaks_pipe_weld_joint_failure_mains,hazardous_leaks_pipe_weld_joint_failure_services,headquarters_address_city,headquarters_address_county,headquarters_address_state,headquarters_address_street,headquarters_address_zip,log,main_1940s_miles,main_1950s_miles,main_1960s_miles,main_1970s_miles,main_1980s_miles,main_1990s_miles,main_2000s_miles,main_2010s_miles,main_2020s_miles,main_abs_2_in_or_less_miles,main_abs_2_to_4_in_miles,main_abs_4_to_8_in_miles,main_abs_8_to_12_in_miles,main_abs_over_12_in_miles,main_abs_total_miles,main_abs_unknown_miles,main_all_materials_2_in_or_less_miles,main_all_materials_2_to_4_in_miles,main_all_materials_4_to_8_in_miles,main_all_materials_8_to_12_in_miles,main_all_materials_over_12_in_miles,main_all_materials_total_miles,main_all_materials_unknown_miles,main_all_time_miles,main_cast_or_wrought_iron_2_in_or_less_miles,main_cast_or_wrought_iron_2_to_4_in_miles,main_cast_or_wrought_iron_4_to_8_in_miles,main_cast_or_wrought_iron_8_to_12_in_miles,main_cast_or_wrought_iron_miles,main_cast_or_wrought_iron_over_12_in_miles,main_cast_or_wrought_iron_total_miles,main_cast_or_wrought_iron_unknown_miles,main_cathodically_protected_steel_bare_miles,main_cathodically_protected_steel_coated_miles,main_copper_2_in_or_less_miles,main_copper_2_to_4_in_miles,main_copper_4_to_8_in_miles,main_copper_8_to_12_in_miles,main_copper_miles,main_copper_over_12_in_miles,main_copper_total_miles,main_copper_unknown_miles,main_ductile_iron_2_in_or_less_miles,main_ductile_iron_2_to_4_in_miles,main_ductile_iron_4_to_8_in_miles,main_ductile_iron_8_to_12_in_miles,main_ductile_iron_miles,main_ductile_iron_over_12_in_miles,main_ductile_iron_total_miles,main_ductile_iron_unknown_miles,main_other_2_in_or_less_miles,main_other_2_to_4_in_miles,main_other_4_to_8_in_miles,main_other_8_to_12_in_miles,main_other_alt_2_in_or_less_miles,main_other_alt_2_to_4_in_miles,main_other_alt_4_to_8_in_miles,main_other_alt_8_to_12_in_miles,main_other_alt_miles,main_other_alt_over_12_in_miles,main_other_alt_total_miles,main_other_alt_unknown_miles,main_other_material_detail_miles,main_other_miles,main_other_over_12_in_miles,main_other_plastic_2_in

In [39]:
# What values are different when we look at AMARILLO NATURAL GAS INC
filtered_df = demand_forecasts[demand_forecasts.operator_id_phmsa==383.0]

columns_with_differences = filtered_df.loc[:, filtered_df.nunique() > 1]

result_dict = {
    col: ", ".join(map(str, columns_with_differences[col].values))
    for col in columns_with_differences.columns
}

# Step 4: Pretty print the dictionary
import pprint
pprint.pprint(result_dict)
sorted_columns = sorted(demand_forecasts.columns)

# Pretty print the sorted columns
for col in sorted_columns:
    print(col)

{'excavation_damage_locating_practices': '0.0, 1.0',
 'excavation_damage_one_call_notification': '1.0, 0.0',
 'excavation_tickets': '11.0, 435.0',
 'filing_date': '2023-01-30T13:25:30.000000000, 2023-01-30T13:18:49.000000000',
 'main_1950s_miles': '0.0, 38.5',
 'main_1960s_miles': '0.0, 29.64',
 'main_1980s_miles': '0.0, 19.72',
 'main_1990s_miles': '9.206, 36.39',
 'main_2000s_miles': '2.38, 4.17',
 'main_2010s_miles': '0.0, 2.02',
 'main_2020s_miles': '0.0, 9.87',
 'main_all_materials_2_in_or_less_miles': '0.0, 16.83',
 'main_all_materials_2_to_4_in_miles': '11.586, 123.48',
 'main_all_materials_total_miles': '11.586, 140.31',
 'main_all_time_miles': '11.586, 140.31',
 'main_pe_2_in_or_less_miles': '0.0, 11.31',
 'main_pe_2_to_4_in_miles': '11.586, 45.51',
 'main_pe_total_miles': '11.586, 56.82',
 'main_plastic_miles': '11.586, 56.82',
 'main_steel_2_in_or_less_miles': '0.0, 5.52',
 'main_steel_2_to_4_in_miles': '0.0, 77.97',
 'main_steel_total_miles': '0.0, 83.49',
 'main_total_mile

In [54]:
sorted_cols = sorted(demand_forecasts.columns)

for col in sorted_cols:
    print(col)

additional_information
all_known_leaks_scheduled_for_repair
all_leaks_construction_defect_mains
all_leaks_construction_defect_services
all_leaks_corrosion_failure_mains
all_leaks_corrosion_failure_services
all_leaks_equipment_failure_mains
all_leaks_equipment_failure_services
all_leaks_excavation_damage_mains
all_leaks_excavation_damage_services
all_leaks_incorrect_operation_mains
all_leaks_incorrect_operation_services
all_leaks_material_defect_mains
all_leaks_material_defect_services
all_leaks_natural_force_damage_mains
all_leaks_natural_force_damage_services
all_leaks_other_mains
all_leaks_other_outside_force_mains
all_leaks_other_outside_force_services
all_leaks_other_services
all_leaks_outside_force_mains
all_leaks_outside_force_services
all_leaks_pipe_weld_joint_failure_mains
all_leaks_pipe_weld_joint_failure_services
all_leaks_third_party_mains
all_leaks_third_party_services
average_service_length_feet
commodity
correction_date
data_date
data_maturity
excavation_damage_excavation

## core_phmsagas__yearly_distribution_operators

In [52]:
core_phmsagas__yearly_distribution_operators = demand_forecasts[[
    "report_date",
    "report_number",
    "report_submission_type",
    "report_year",
    "operator_id_phmsa",
    "operator_name_phmsa",
    "office_address_street",
    "office_address_city",
    "office_address_state",
    "office_address_zip",
    "office_address_county",
    "operator_id_phmsa",
    "excavation_damage_excavation_practices",
    "excavation_damage_locating_practices",
    "excavation_damage_one_call_notification",
    "excavation_damage_other",
    "excavation_damage_total",
    "excavation_tickets",
    "services_efv_in_system",
    "services_efv_installed",
    "services_shutoff_valve_in_system",
    "services_shutoff_valve_installed",
    "federal_land_leaks_repaired_or_scheduled",
    "percent_unaccounted_for_gas",
    "additional_information",
    "preparer_email",
    "preparer_fax",
    "preparer_name",
    "preparer_phone",
    "preparer_title"
]]

core_phmsagas__yearly_distribution_operators.head()

,report_date,report_number,report_submission_type,report_year,operator_id_phmsa,operator_name_phmsa,office_address_street,office_address_city,office_address_state,office_address_zip,office_address_county,operator_id_phmsa,excavation_damage_excavation_practices,excavation_damage_locating_practices,excavation_damage_one_call_notification,excavation_damage_other,excavation_damage_total,excavation_tickets,services_efv_in_system,services_efv_installed,services_shutoff_valve_in_system,services_shutoff_valve_installed,federal_land_leaks_repaired_or_scheduled,percent_unaccounted_for_gas,additional_information,preparer_email,preparer_fax,preparer_name,preparer_phone,preparer_title
0,2023-03-06 16:30:43,20230656.0,INITIAL,2022.0,18.0,"ABBYVILLE, CITY OF",PO BOX 100,ABBYVILLE,KS,67510,Reno,18.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.00,NaN,(620) 500-5758,cityofabbyville@hotmail.com,Mindy Iverson,(620) 286-5643,agent
1,2023-02-15 13:07:14,20230299.0,INITIAL,2022.0,27.0,ABITA SPRINGS NAT GAS & WATER,222 Highway 19,SLAUGHTER,LA,70777-3521,NaN,27.0,2.0,0.0,1.0,0.0,3.0,1047.0,133.0,27.0,25.0,0.0,0.0,3.90,NaN,(000) 000-0000,kewing.pip@gmail.com,Kaylan A. Ewing,(225) 286-5650,Consultant
2,2023-03-15 14:08:48,20231215.0,INITIAL,2022.0,45.0,"ADAIRSVILLE, CITY OF",2369 Hall Station Rd NW,Adairsville,GA,30103,Bartow,45.0,NaN,1.0,NaN,NaN,1.0,1815.0,1292.0,68.0,2.0,0.0,0.0,-1.60,This report prepared by: Maurice Chaney Munici...,(678) 819-0957,mchaney@gasauthority.com,MAURICE CHANEY,(770) 851-3094,Consultant
3,2023-03-06 11:55:36,20230639.0,INITIAL,2022.0,49.0,TOWN OF ADAMSVILLE GAS DEPT,203 Sunrise Drive 231 East Main Street,Adamsville,TN,38310,McNairy,49.0,8.0,5.0,2.0,0.0,15.0,963.0,671.0,7.0,8.0,3.0,0.0,5.42,We have stopped using the radio read transpond...,(000) 000-0000,scottklinck@yahoo.com,Scott Klinck,(731) 632-4214,Publc Works Director
4,2023-02-14 15:41:24,20230289.0,INITIAL,2022.0,54.0,"ADEL GAS DEPT, CITY OF",P.O. Box 1530,Adel,GA,31620,Cook,54.0,0.0,0.0,0.0,0.0,0.0,448.0,102.0,3.0,2.0,0.0,0.0,0.00,NaN,(000) 000-0000,peterschultzharpassociates@yahoo.com,Peter Schultz,(229) 834-6246,agent


## Operator Table Tasks

Next part to complete